## Exercises

Using the [repo setup directions](https://ds.codeup.com/fundamentals/git/), setup a new local and remote repository named `spark-exercises`. The local version of your repo should live inside of `~/codeup-data-science`. This repo should be named `spark-exercises`

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

In [1]:
import pandas as pd
import numpy as np
import pyspark

1. Create a spark data frame that contains your favorite programming languages.

In [2]:
# in order to work with spark dataframes,
# we will need to establish a spark session
session = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/26 09:30:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
session

    - The name of the column should be `language`
    

In [4]:
fav_langs = session.createDataFrame(pd.DataFrame(
{
    'language': [
        'python',
        'r',
        'c',
        'rust',
        'ruby',
        'c++',
        'java',
        'golang'
    ]
    
}))

    - View the schema of the dataframe

In [5]:
fav_langs.printSchema()

root
 |-- language: string (nullable = true)



    - Output the shape of the dataframe

In [6]:
# take the count of rows, followed by the length of columns
fav_langs.count(), len(fav_langs.columns)

(8, 1)

     - Show the first 5 records in the dataframe

In [7]:
fav_langs.show(5)

+--------+
|language|
+--------+
|  python|
|       r|
|       c|
|    rust|
|    ruby|
+--------+
only showing top 5 rows



2. Load the `mpg` dataset as a spark dataframe.

In [8]:
from pydataset import data
mpg = session.createDataFrame(data('mpg'))

    - Create 1 column of output that contains a message like the one below:

            The 1999 audi a4 has a 4 cylinder engine.

        For each vehicle.

In [9]:
from pyspark.sql import functions as F

In [10]:
# f.lit() casting on string literals in a f.concat()
mpg.select(
F.concat(F.lit('The '),
mpg.year,
F.lit(' '),
mpg.manufacturer,
F.lit(' '),
mpg.model,
F.lit(' has a '),
mpg.cyl,
F.lit(' cylinder engine.')).alias('car_sentence')).show(truncate=False)


+--------------------------------------------------------------+
|car_sentence                                                  |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

     - Transform the `trans` column so that it only contains either `manual` or `auto`.

In [11]:
# previously in pandas:
#np.where(mpg.trans.str.startswith('auto'), 'automatic', 'manual')
# to do this in pyspark:
mpg.select(
F.when(
    F.col('trans').startswith('auto'),
'auto').otherwise('manual').alias('transmission_custom')).show(5)

+-------------------+
|transmission_custom|
+-------------------+
|               auto|
|             manual|
|             manual|
|               auto|
|               auto|
+-------------------+
only showing top 5 rows



In [12]:
mpg.select(
F.when(
    F.expr("trans LIKE 'auto%'"),
'auto').otherwise('manual').alias('transmission_custom')).show(5)

+-------------------+
|transmission_custom|
+-------------------+
|               auto|
|             manual|
|             manual|
|               auto|
|               auto|
+-------------------+
only showing top 5 rows



3. Load the `tips` dataset as a spark dataframe.

    1. What percentage of observations are smokers?
    

In [13]:
# load up tips from pydataset, feed it into createDataFrame
tips = session.createDataFrame(data('tips'))

In [14]:
# group by smoker column,
# grab the counts of each subpopulation,
# make a new column (withColumn) called percent
# reference the new aggreagted column count, divide by the length of the df
# multiply by 100 to get the percentage, round the whole thing
# then show
tips.groupBy('smoker').agg(
    F.round(
        (F.count(F.col('smoker')
                ) / tips.count()
        ) * 100, 2).alias('percent_smoker')
).show()

+------+--------------+
|smoker|percent_smoker|
+------+--------------+
|    No|         61.89|
|   Yes|         38.11|
+------+--------------+



    2. Create a column that contains the tip percentage
    

In [15]:
tips.withColumn(
    'percent_tip',
    F.round((tips.tip / tips.total_bill) * 100, 2)).select(
    'total_bill', 'tip','percent_tip').show(5)

+----------+----+-----------+
|total_bill| tip|percent_tip|
+----------+----+-----------+
|     16.99|1.01|       5.94|
|     10.34|1.66|      16.05|
|     21.01| 3.5|      16.66|
|     23.68|3.31|      13.98|
|     24.59|3.61|      14.68|
+----------+----+-----------+
only showing top 5 rows



    3. Calculate the average tip percentage for each combination of sex and smoker.

In [16]:
tips.withColumn(
    'percent_tip',
    F.round((tips.tip / tips.total_bill) * 100, 2)
).groupBy('sex', 'smoker').agg(
    F.round(
        F.mean(
            F.col('percent_tip')
        ),2
    ).alias('mean_group_tip')
).show()

+------+------+--------------+
|   sex|smoker|mean_group_tip|
+------+------+--------------+
|  Male|    No|         16.07|
|Female|    No|         15.69|
|  Male|   Yes|         15.28|
|Female|   Yes|         18.21|
+------+------+--------------+



In [17]:
# pivot table version:
tips.withColumn(
    'percent_tip',
    F.round((tips.tip / tips.total_bill) * 100, 2)
).groupby('sex').pivot('smoker').agg(
    F.round(
        F.mean(
            F.col('percent_tip')
        ),2
    ).alias('mean_group_tip')
).show()

+------+-----+-----+
|   sex|   No|  Yes|
+------+-----+-----+
|Female|15.69|18.21|
|  Male|16.07|15.28|
+------+-----+-----+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

    - Convert the temperatures to fahrenheit.
    

In [18]:
from vega_datasets import data
weather = data.seattle_weather()
weather = session.createDataFrame(weather)

In [19]:
# c to f: (0°C × 9/5) + 32 = 32°F

In [20]:
weather.show(1)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
+-------------------+-------------+--------+--------+----+-------+
only showing top 1 row



In [21]:
# create the new weather columns
# drop the old ones
weather = weather.withColumns(
{
    'max_temp': F.round(F.col('temp_max') * (9/5) + 32, 2),
    'min_temp': F.round(F.col('temp_min') * (9/5) + 32, 2)
}).drop('temp_max', 'temp_min')

    - Which month has the most rain, on average?
    

In [22]:
# extract the month from the date
# calculate average rain for each month in aggregate
# sort the dataframe descending
weather.groupBy(
    F.month(
        F.col('date')
    )
).agg(
    F.round(
        F.avg(
            F.col(
                'precipitation')
        ), 2).alias('avg_rain')).sort(F.col('avg_rain'),
                                      ascending=False).first()

Row(month(date)=11, avg_rain=5.35)

    - Which year was the windiest?
    

In [23]:
# weather.show(1)

In [24]:
weather.groupBy(
    F.year(
        F.col('date')
    )
).agg(
    F.round(
        F.avg(
            F.col(
                'wind')
        ), 2).alias('avg_wind')).sort(F.col('avg_wind'),
                                      ascending=False).first()

Row(year(date)=2012, avg_wind=3.4)

    - What is the most frequent type of weather in January?
    

In [25]:
# weather in january, what weather type pops up the most?
# aggregate on the month of january,
# get the count of weather types where its the maximum

In [26]:
weather.filter(
    F.month(weather.date) == 1
).groupBy(
    F.col('weather')
).count().sort(F.col('count'), ascending=False).first()

Row(weather='fog', count=38)

    - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    

In [27]:
# want to see: avg high and avg low
# conditions:
# weather == sun
# month == 7
# year in 2013 or 2014

In [28]:
weather.filter(
    (
        F.col('weather'
             ) == 'sun'
    ) & (
        F.month(F.col('date')
               ) == 7
    )
).filter(
    F.year(
        F.col('date')).isin(2013, 2014)).agg(
    F.round(
        F.avg(
            F.col('max_temp')),2).alias('avg_max'),
    F.round(
        F.avg(
            F.col('min_temp')),2).alias('avg_min')).show()

+-------+-------+
|avg_max|avg_min|
+-------+-------+
|  80.29|  57.53|
+-------+-------+



    - What percentage of days were rainy in q3 of 2015?
    

In [29]:
# conditions:
# quarter = 3
# year == 2015
# percent of days where it rained
weather.filter(
(F.quarter(F.col('date')) == 3) &
(F.year(F.col('date')) == 2015)
).withColumn(
    'rain_happen',
    F.when(
        F.col('weather') == F.lit('rain'),1).otherwise(0)
).agg(F.round(F.mean(F.col('rain_happen')) * 100,2).alias('avg_rain_days')).show()

+-------------+
|avg_rain_days|
+-------------+
|         2.17|
+-------------+



    - For each year, find what percentage of days it rained (had non-zero precipitation).

In [30]:
weather

DataFrame[date: timestamp, precipitation: double, wind: double, weather: string, max_temp: double, min_temp: double]

In [31]:
# how many days did it rain?
# the percentage of days that have non-zero precipitation
# Building out a strategy for this process:
# 1 - find out days that have had more than zero precip, which
# will result in a boolean value
# that sounds like a setup for a 
# F.when(#condition ==> did_rain bool)
# once I have did_rain rendered as a boolean, I know that if I take the mean
# of said boolean I will result with a rate as a result of the number of true instances
# over the denominator of the number of rows in the df.
# I want to know this for each year, so this process should happen in the scope
# of an aggregation for year-by-year.

In [32]:
# things I know I will need:
# a groupby
# potentially a when statement*
weather.withColumn(
    'rain_happen',
    F.when(
        F.col('precipitation') > 0,
        1).otherwise(0)
).select('precipitation', 'rain_happen').show(5)

+-------------+-----------+
|precipitation|rain_happen|
+-------------+-----------+
|          0.0|          0|
|         10.9|          1|
|          0.8|          1|
|         20.3|          1|
|          1.3|          1|
+-------------+-----------+
only showing top 5 rows



In [38]:
# alternate method for grabbing truth values
# of did_rain: make the boolean comparison, cast as an integer
weather.withColumn(
    'rain_happen',
    (weather.precipitation > 0).cast('int')
).select('precipitation', 'rain_happen').show(5)

+-------------+-----------+
|precipitation|rain_happen|
+-------------+-----------+
|          0.0|          0|
|         10.9|          1|
|          0.8|          1|
|         20.3|          1|
|          1.3|          1|
+-------------+-----------+
only showing top 5 rows



In [40]:
# lets pick one of the previous code blocks and run with it
# to proceed forward with our yearly aggregation
weather.withColumn(
    'rain_happen',
    (weather.precipitation > 0).cast('int')
).groupBy(
    F.year(
        F.col(
            'date')
    )
).agg(
    F.round(F.mean('rain_happen') * 100, 2).alias('rain_percent')
).show()

+----------+------------+
|year(date)|rain_percent|
+----------+------------+
|      2012|       48.36|
|      2013|       41.64|
|      2014|        41.1|
|      2015|       39.45|
+----------+------------+

